In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import seaborn as sns
import re

%matplotlib inline 

In [5]:
member_df = pd.read_csv('../data/NSS_GreatCare_Members.csv')
member_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61519 entries, 0 to 61518
Data columns (total 63 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Patient ID                                                       61519 non-null  object 
 1   Primary Plan ID                                                  61519 non-null  object 
 2   Primary Payor ID                                                 61519 non-null  object 
 3   Is Patient Alive                                                 61519 non-null  int64  
 4   Patient Age Integer                                              61519 non-null  int64  
 5   Patient Date of Birth                                            61519 non-null  object 
 6   Patient ZIP Code 5-Digit                                         61105 non-null  float64
 7   Count Primary Care Visits in Last 36 Mon

In [6]:
inpatient_admin = pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv')
inpatient_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Encounter ID                                         6213 non-null   object 
 1   Patient ID                                           6213 non-null   object 
 2   Encounter Primary Payor ID                           6213 non-null   object 
 3   Encounter Primary Plan ID                            6213 non-null   object 
 4   Facility Provider ID                                 6213 non-null   object 
 5   Attending Provider ID                                5884 non-null   object 
 6   Attending Provider NPI                               5884 non-null   float64
 7   Facility Provider NPI                                6213 non-null   object 
 8   Encounter Start Date                                 6213 non-null  

In [15]:
# Converting some data types, specifically the dates

inpatient_admin[['Encounter Start Date','Encounter End Date']] = inpatient_admin[['Encounter Start Date','Encounter End Date']].apply(pd.to_datetime)
inpatient_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   Encounter ID                                         6213 non-null   object        
 1   Patient ID                                           6213 non-null   object        
 2   Encounter Primary Payor ID                           6213 non-null   object        
 3   Encounter Primary Plan ID                            6213 non-null   object        
 4   Facility Provider ID                                 6213 non-null   object        
 5   Attending Provider ID                                5884 non-null   object        
 6   Attending Provider NPI                               5884 non-null   float64       
 7   Facility Provider NPI                                6213 non-null   object        
 8 

In [8]:
pcp_visit = pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv',low_memory = False)
pcp_visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231373 entries, 0 to 231372
Data columns (total 18 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   Encounter ID                                   231373 non-null  object 
 1   Patient ID                                     231373 non-null  object 
 2   Service Provider ID                            230599 non-null  object 
 3   Attending Provider ID                          39644 non-null   object 
 4   Provider Group Provider ID                     227644 non-null  object 
 5   Facility Provider ID                           39756 non-null   object 
 6   Encounter Primary Payor ID                     231373 non-null  object 
 7   Encounter Primary Plan ID                      231373 non-null  object 
 8   Attending Provider NPI                         39644 non-null   float64
 9   Facility Provider NPI                

In [16]:
pcp_visit['Encounter Date'] = pd.to_datetime(pcp_visit['Encounter Date'])
pcp_visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231373 entries, 0 to 231372
Data columns (total 18 columns):
 #   Column                                         Non-Null Count   Dtype         
---  ------                                         --------------   -----         
 0   Encounter ID                                   231373 non-null  object        
 1   Patient ID                                     231373 non-null  object        
 2   Service Provider ID                            230599 non-null  object        
 3   Attending Provider ID                          39644 non-null   object        
 4   Provider Group Provider ID                     227644 non-null  object        
 5   Facility Provider ID                           39756 non-null   object        
 6   Encounter Primary Payor ID                     231373 non-null  object        
 7   Encounter Primary Plan ID                      231373 non-null  object        
 8   Attending Provider NPI                      

In [25]:
# Going to start on question 2 cause I think that's easier

# Gonna pick the columns I need from 'Inpatient' and 'PCP' and then join them

x = inpatient_admin[['Encounter ID','Patient ID', 'Encounter Start Date', 'Encounter End Date']]
y = pcp_visit[['Encounter ID','Patient ID', 'Encounter Date']]

print(x)
print(y)

                            Encounter ID Patient ID Encounter Start Date  \
0     HIA|DS1|10033|DS1|NPI|1306890389|1  DS1|10033           2013-01-12   
1     HIA|DS1|10043|DS1|NPI|1902803315|1  DS1|10043           2014-01-05   
2     HIA|DS1|10073|DS1|NPI|1902803315|1  DS1|10073           2012-06-13   
3     HIA|DS1|10101|DS1|NPI|1427055839|1  DS1|10101           2012-12-06   
4     HIA|DS1|10118|DS1|NPI|1992818256|1  DS1|10118           2015-03-11   
...                                  ...        ...                  ...   
6208   HIA|DS1|9979|DS1|NPI|1689772592|1   DS1|9979           2015-02-06   
6209   HIA|DS1|9981|DS1|NPI|1427055839|1   DS1|9981           2014-08-07   
6210   HIA|DS1|9994|DS1|NPI|1689608150|1   DS1|9994           2015-11-21   
6211   HIA|DS1|9994|DS1|NPI|1689608150|2   DS1|9994           2015-11-24   
6212   HIA|DS1|9996|DS1|NPI|1033298617|1   DS1|9996           2010-03-28   

     Encounter End Date  
0            2013-01-14  
1            2014-01-12  
2        

In [26]:
# Merge. There will be duplicates because of the mismatch dates between PCP visits and Inpatient Visits.

inpatient_pcp_df = x.merge(y, how = 'left', on = 'Patient ID')
inpatient_pcp_df

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1228384|1043427248|1|SHIFTED,2015-12-08
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1326684|1043427248|1|SHIFTED,2012-06-18
2,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|155317|1043427248|1|SHIFTED,2014-06-10
3,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|439093|1043427248|1|SHIFTED,2014-09-28
4,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|711310|1043427248|1|SHIFTED,2014-03-16
...,...,...,...,...,...,...
44093,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|1238517|1407058787|2,2015-12-05
44094,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|302749|1407058787|2,2015-12-13
44095,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|781849|1205884392|1,2014-05-18
44096,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|93182|1043427248|1,2013-07-20


In [28]:
# Time to find if they visited the pcp after hospital

inpatient_pcp_df['7_Days'] = inpatient_pcp_df['Encounter Date'] - inpatient_pcp_df['Encounter Start Date']
inpatient_pcp_df

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date,7_Days
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1228384|1043427248|1|SHIFTED,2015-12-08,1060 days
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1326684|1043427248|1|SHIFTED,2012-06-18,-208 days
2,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|155317|1043427248|1|SHIFTED,2014-06-10,514 days
3,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|439093|1043427248|1|SHIFTED,2014-09-28,624 days
4,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|711310|1043427248|1|SHIFTED,2014-03-16,428 days
...,...,...,...,...,...,...,...
44093,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|1238517|1407058787|2,2015-12-05,11 days
44094,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|302749|1407058787|2,2015-12-13,19 days
44095,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|781849|1205884392|1,2014-05-18,-555 days
44096,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|93182|1043427248|1,2013-07-20,-857 days


In [35]:
t = []

for i in inpatient_pcp_df['7_Days']:
    if (i <= dt.timedelta(days=7)) and (i > dt.timedelta(days=0)) :
        t.append('yes')
    else:
        t.append('no')

In [36]:
inpatient_pcp_df['within_7'] = t

In [38]:
# There are 1607 instances in which a patient visited their PCP within 7 days of an Inpatient visit. 

inpatient_pcp_df['within_7'].value_counts()

no     42491
yes     1607
Name: within_7, dtype: int64

In [ ]:
# Good stopping point, need to finish Q1 and Q3.